In [1]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader, random_split
from conversion import CSVToTensor

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input_size = 9
        self.output_size = 9
        self.hidden_size = 2*27

        self.W1 = nn.Parameter(torch.randn(self.input_size, self.hidden_size))
        self.b1 = nn.Parameter(torch.randn(self.hidden_size))
        self.W2 = nn.Parameter(torch.randn(self.hidden_size, self.output_size))
        self.b2 = nn.Parameter(torch.randn(self.output_size))
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.3)
        self.crossloss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.train_data = None
        self.val_data = None
    
    def forward(self, x):
        x = x @ self.W1 + self.b1
        x = self.relu(x)
        x = x @ self.W2 + self.b2
        # x = self.dropout(x)
        return x

    def load_data(self, file_path):
        dataloader = CSVToTensor(file_path)
        dataloader.create_all_tensor()
        dataset = dataloader.create_a_dataset()

        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        self.train_data, self.val_data = random_split(dataset, [train_size, val_size])
        self.train_data = DataLoader(self.train_data, batch_size=16, shuffle=True)
        self.val_data = DataLoader(self.val_data, batch_size=16, shuffle=True)

    def train_model(self, epochs):
        self.to(self.device)
        self.train()
        for epoch in range(epochs):
            epoch_loss = 0
            for src, trg in self.train_data:
                src = src.to(self.device)
                trg = trg.to(self.device)
                self.optimizer.zero_grad()
                output = self.forward(src)
                loss = self.crossloss(output, trg.argmax(dim=1))
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            avg_loss = epoch_loss / len(self.train_data)
            print(f"Epoch: {epoch}, Loss: {avg_loss:.4f}")

    

In [2]:
if __name__ == '__main__':
    model = Model()
    # choose the dataset file path
    model.load_data('./Datasets/tic_tac_toe_500_games.csv')
    # choose the number of epochs
    model.train_model(100)

Epoch: 0, Loss: 22.3650
Epoch: 1, Loss: 15.7902
Epoch: 2, Loss: 12.7127
Epoch: 3, Loss: 10.5604
Epoch: 4, Loss: 8.7993
Epoch: 5, Loss: 7.3441
Epoch: 6, Loss: 6.1958
Epoch: 7, Loss: 5.3291
Epoch: 8, Loss: 4.6752
Epoch: 9, Loss: 4.2066
Epoch: 10, Loss: 3.8311
Epoch: 11, Loss: 3.5471
Epoch: 12, Loss: 3.3302
Epoch: 13, Loss: 3.1392
Epoch: 14, Loss: 2.9942
Epoch: 15, Loss: 2.8843
Epoch: 16, Loss: 2.7916
Epoch: 17, Loss: 2.7061
Epoch: 18, Loss: 2.6300
Epoch: 19, Loss: 2.5650
Epoch: 20, Loss: 2.4982
Epoch: 21, Loss: 2.4514
Epoch: 22, Loss: 2.3956
Epoch: 23, Loss: 2.3477
Epoch: 24, Loss: 2.2965
Epoch: 25, Loss: 2.2561
Epoch: 26, Loss: 2.2237
Epoch: 27, Loss: 2.1865
Epoch: 28, Loss: 2.1483
Epoch: 29, Loss: 2.1285
Epoch: 30, Loss: 2.0942
Epoch: 31, Loss: 2.0586
Epoch: 32, Loss: 2.0334
Epoch: 33, Loss: 2.0208
Epoch: 34, Loss: 1.9914
Epoch: 35, Loss: 1.9719
Epoch: 36, Loss: 1.9529
Epoch: 37, Loss: 1.9273
Epoch: 38, Loss: 1.9120
Epoch: 39, Loss: 1.8984
Epoch: 40, Loss: 1.8799
Epoch: 41, Loss: 1.862